In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, KFold
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")

In [2]:
Forest_Fires = pd.read_csv('D:/DS Assignments/NN/forestfires.csv')

In [3]:
Forest_Fires.head()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small


In [4]:
label_encoder = preprocessing.LabelEncoder()
Forest_Fires['size_category']= label_encoder.fit_transform(Forest_Fires['size_category'])

In [5]:
Forest_Fires.head()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,1
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,1
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,1
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,1
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,1


In [6]:
X = Forest_Fires.iloc[:,2:30].values

In [7]:
X.shape

(517, 28)

In [8]:
a = StandardScaler()
a.fit(X)
X = a.transform(X)

In [9]:
pd.DataFrame(X).describe()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
count,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,...,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02
mean,-1.754024e-15,3.070830e-16,7.387171e-17,-3.865380e-17,2.005703e-16,3.362881e-16,-2.676776e-16,-2.841054e-16,-1.274502e-16,4.874674e-17,...,7.179943e-16,-1.933764e-16,-2.260174e-17,1.352883e-17,1.169277e-16,2.265542e-16,-2.596515e-16,1.443075e-16,6.253326e-16,4.024290e-16
std,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,...,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00
min,-1.304582e+01,-1.715608e+00,-2.179108e+00,-1.980578e+00,-2.876943e+00,-1.796637e+00,-2.021098e+00,-7.326831e-02,-2.020198e-01,-4.435755e-01,...,-1.331035e-01,-2.006027e-01,-6.231770e-02,-2.568645e-01,-1.843909e-01,-3.415123e-01,-6.231770e-02,-4.402255e-02,-1.728597e-01,-7.060812e-01
25%,-8.063453e-02,-6.606652e-01,-4.448281e-01,-5.535954e-01,-5.842379e-01,-6.924563e-01,-7.361236e-01,-7.326831e-02,-2.020198e-01,-4.435755e-01,...,-1.331035e-01,-2.006027e-01,-6.231770e-02,-2.568645e-01,-1.843909e-01,-3.415123e-01,-6.231770e-02,-4.402255e-02,-1.728597e-01,-7.060812e-01
50%,1.732292e-01,-4.020255e-02,4.691190e-01,-1.364774e-01,7.082076e-02,-1.403660e-01,-9.833712e-03,-7.326831e-02,-1.938429e-01,-4.435755e-01,...,-1.331035e-01,-2.006027e-01,-6.231770e-02,-2.568645e-01,-1.843909e-01,-3.415123e-01,-6.231770e-02,-4.402255e-02,-1.728597e-01,-7.060812e-01
75%,4.089598e-01,4.927389e-01,6.696628e-01,3.904086e-01,6.741643e-01,5.344111e-01,4.929823e-01,-7.326831e-02,-9.870852e-02,-4.435755e-01,...,-1.331035e-01,-2.006027e-01,-6.231770e-02,-2.568645e-01,-1.843909e-01,-3.415123e-01,-6.231770e-02,-4.402255e-02,-1.728597e-01,1.416268e+00
max,1.007353e+00,2.819865e+00,1.261610e+00,1.033538e+01,2.484195e+00,3.417549e+00,3.007063e+00,2.157228e+01,1.695111e+01,2.254407e+00,...,7.512952e+00,4.984977e+00,1.604681e+01,3.893103e+00,5.423261e+00,2.928152e+00,1.604681e+01,2.271563e+01,5.785038e+00,1.416268e+00


In [10]:
pca = PCA(n_components = 20)
X1 = pca.fit_transform(X)

In [11]:
var = pca.explained_variance_ratio_
var

array([0.13552275, 0.06857888, 0.06235727, 0.05327133, 0.04759424,
       0.04680099, 0.043749  , 0.04280252, 0.04088757, 0.04016333,
       0.03929269, 0.03832323, 0.03642215, 0.03632173, 0.03578568,
       0.03500878, 0.03354477, 0.03247774, 0.03044909, 0.03002468])

In [12]:
var1 = np.cumsum(np.round(var,decimals = 4)*100)
var1

array([13.55, 20.41, 26.65, 31.98, 36.74, 41.42, 45.79, 50.07, 54.16,
       58.18, 62.11, 65.94, 69.58, 73.21, 76.79, 80.29, 83.64, 86.89,
       89.93, 92.93])

In [13]:
y = np.ravel(Forest_Fires.size_category)

In [14]:
y.shape

(517,)

In [15]:
# create model
model = Sequential()
model.add(Flatten())
model.add(Dense(12, input_dim=20, kernel_initializer='uniform', activation='relu'))
model.add(Dense(8, kernel_initializer ='uniform', activation='tanh'))
model.add(Dense(1, kernel_initializer ='uniform', activation='softmax'))

In [18]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
# Fit the model
model.fit(X1, y, validation_split=0.30, epochs=150, batch_size=10)

Epoch 1/150
37/37 [==============================] - 1s 6ms/step - loss: 0.6874 - accuracy: 0.7562 - val_loss: 0.6833 - val_accuracy: 0.6731
Epoch 2/150
37/37 [==============================] - 0s 2ms/step - loss: 0.6639 - accuracy: 0.7562 - val_loss: 0.6610 - val_accuracy: 0.6731
Epoch 3/150
37/37 [==============================] - 0s 2ms/step - loss: 0.6188 - accuracy: 0.7562 - val_loss: 0.6336 - val_accuracy: 0.6731
Epoch 4/150
37/37 [==============================] - 0s 2ms/step - loss: 0.5651 - accuracy: 0.7562 - val_loss: 0.6202 - val_accuracy: 0.6731
Epoch 5/150
37/37 [==============================] - 0s 2ms/step - loss: 0.5265 - accuracy: 0.7562 - val_loss: 0.6209 - val_accuracy: 0.6731
Epoch 6/150
37/37 [==============================] - 0s 2ms/step - loss: 0.5079 - accuracy: 0.7562 - val_loss: 0.6207 - val_accuracy: 0.6731
Epoch 7/150
37/37 [==============================] - 0s 2ms/step - loss: 0.4941 - accuracy: 0.7562 - val_loss: 0.6221 - val_accuracy: 0.6731
Epoch 8/150
3

Epoch 59/150
37/37 [==============================] - 0s 2ms/step - loss: 0.1960 - accuracy: 0.7562 - val_loss: 0.4616 - val_accuracy: 0.6731
Epoch 60/150
37/37 [==============================] - 0s 2ms/step - loss: 0.1997 - accuracy: 0.7562 - val_loss: 0.4609 - val_accuracy: 0.6731
Epoch 61/150
37/37 [==============================] - 0s 2ms/step - loss: 0.1890 - accuracy: 0.7562 - val_loss: 0.4632 - val_accuracy: 0.6731
Epoch 62/150
37/37 [==============================] - 0s 2ms/step - loss: 0.1842 - accuracy: 0.7562 - val_loss: 0.4646 - val_accuracy: 0.6731
Epoch 63/150
37/37 [==============================] - 0s 2ms/step - loss: 0.1856 - accuracy: 0.7562 - val_loss: 0.4629 - val_accuracy: 0.6731
Epoch 64/150
37/37 [==============================] - 0s 2ms/step - loss: 0.1857 - accuracy: 0.7562 - val_loss: 0.4578 - val_accuracy: 0.6731
Epoch 65/150
37/37 [==============================] - 0s 2ms/step - loss: 0.1800 - accuracy: 0.7562 - val_loss: 0.4704 - val_accuracy: 0.6731
Epoch 

In [20]:
scores = model.evaluate(X1, y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

17/17 [==============================] - 0s 1ms/step - loss: 0.2388 - accuracy: 0.7311
accuracy: 73.11%


# Tuning of Hyperparameters :- Batch Size and Epochs

In [21]:
# create model
def create_model():
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(12, input_dim=20, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='softmax'))
    
    adam=Adam(lr=0.01)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [22]:
# Create the model
model = KerasClassifier(build_fn = create_model,verbose = 0)
# Define the grid search parameters
batch_size = [10,20,40]
epochs = [10,50,100]
# Make a dictionary of the grid search parameters
param_grid = dict(batch_size = batch_size,epochs = epochs)
# Build and fit the GridSearchCV
grid = GridSearchCV(estimator = model,param_grid = param_grid,cv = KFold(),verbose = 10)
grid_result = grid.fit(X1,y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START batch_size=10, epochs=10....................................
[CV 1/5; 1/9] END ..................batch_size=10, epochs=10; total time=   0.9s
[CV 2/5; 1/9] START batch_size=10, epochs=10....................................
[CV 2/5; 1/9] END ..................batch_size=10, epochs=10; total time=   0.9s
[CV 3/5; 1/9] START batch_size=10, epochs=10....................................
[CV 3/5; 1/9] END ..................batch_size=10, epochs=10; total time=   0.9s
[CV 4/5; 1/9] START batch_size=10, epochs=10....................................
[CV 4/5; 1/9] END ..................batch_size=10, epochs=10; total time=   0.9s
[CV 5/5; 1/9] START batch_size=10, epochs=10....................................
[CV 5/5; 1/9] END ..................batch_size=10, epochs=10; total time=   0.9s
[CV 1/5; 2/9] START batch_size=10, epochs=50....................................
[CV 1/5; 2/9] END ..................batch_size=10

[CV 2/5; 9/9] END .................batch_size=40, epochs=100; total time=   1.1s
[CV 3/5; 9/9] START batch_size=40, epochs=100...................................
[CV 3/5; 9/9] END .................batch_size=40, epochs=100; total time=   1.4s
[CV 4/5; 9/9] START batch_size=40, epochs=100...................................
[CV 4/5; 9/9] END .................batch_size=40, epochs=100; total time=   1.2s
[CV 5/5; 9/9] START batch_size=40, epochs=100...................................
[CV 5/5; 9/9] END .................batch_size=40, epochs=100; total time=   1.2s


In [23]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.7305825233459473, using {'batch_size': 10, 'epochs': 10}
0.7305825233459473,0.15435061319000673 with: {'batch_size': 10, 'epochs': 10}
0.7305825233459473,0.15435061319000673 with: {'batch_size': 10, 'epochs': 50}
0.7305825233459473,0.15435061319000673 with: {'batch_size': 10, 'epochs': 100}
0.7305825233459473,0.15435061319000673 with: {'batch_size': 20, 'epochs': 10}
0.7305825233459473,0.15435061319000673 with: {'batch_size': 20, 'epochs': 50}
0.7305825233459473,0.15435061319000673 with: {'batch_size': 20, 'epochs': 100}
0.7305825233459473,0.15435061319000673 with: {'batch_size': 40, 'epochs': 10}
0.7305825233459473,0.15435061319000673 with: {'batch_size': 40, 'epochs': 50}
0.7305825233459473,0.15435061319000673 with: {'batch_size': 40, 'epochs': 100}


In [16]:
def create_model1(neuron1,neuron2):
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(neuron1,input_dim = 20,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = 'normal',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'softmax'))
    
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

In [18]:
# Create the model

model1 = KerasClassifier(build_fn = create_model1,verbose = 0)

In [19]:
# Define the grid search parameters

batch_size = [10,20,40]
epochs = [10,50,100]
neuron1 = [4,8,16]
neuron2 = [2,4,8]
#Make a dictionary of the grid search parameters

param_grids = dict(batch_size = batch_size,epochs = epochs,neuron1 = neuron1,neuron2 = neuron2)
# Build and fit the GridSearchCV

grid1 = GridSearchCV(estimator = model1,param_grid = param_grids,cv = KFold(),verbose = 10)

In [20]:
grid_result1 = grid1.fit(X1,y)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV 1/5; 1/81] START batch_size=10, epochs=10, neuron1=4, neuron2=2.............
[CV 1/5; 1/81] END batch_size=10, epochs=10, neuron1=4, neuron2=2; total time=   1.3s
[CV 2/5; 1/81] START batch_size=10, epochs=10, neuron1=4, neuron2=2.............
[CV 2/5; 1/81] END batch_size=10, epochs=10, neuron1=4, neuron2=2; total time=   1.1s
[CV 3/5; 1/81] START batch_size=10, epochs=10, neuron1=4, neuron2=2.............
[CV 3/5; 1/81] END batch_size=10, epochs=10, neuron1=4, neuron2=2; total time=   1.0s
[CV 4/5; 1/81] START batch_size=10, epochs=10, neuron1=4, neuron2=2.............
[CV 4/5; 1/81] END batch_size=10, epochs=10, neuron1=4, neuron2=2; total time=   1.0s
[CV 5/5; 1/81] START batch_size=10, epochs=10, neuron1=4, neuron2=2.............
[CV 5/5; 1/81] END batch_size=10, epochs=10, neuron1=4, neuron2=2; total time=   1.0s
[CV 1/5; 2/81] START batch_size=10, epochs=10, neuron1=4, neuron2=4.............
[CV 1/5; 2/81] END bat

[CV 5/5; 10/81] END batch_size=10, epochs=50, neuron1=4, neuron2=2; total time=   1.9s
[CV 1/5; 11/81] START batch_size=10, epochs=50, neuron1=4, neuron2=4............
[CV 1/5; 11/81] END batch_size=10, epochs=50, neuron1=4, neuron2=4; total time=   1.9s
[CV 2/5; 11/81] START batch_size=10, epochs=50, neuron1=4, neuron2=4............
[CV 2/5; 11/81] END batch_size=10, epochs=50, neuron1=4, neuron2=4; total time=   1.9s
[CV 3/5; 11/81] START batch_size=10, epochs=50, neuron1=4, neuron2=4............
[CV 3/5; 11/81] END batch_size=10, epochs=50, neuron1=4, neuron2=4; total time=   2.0s
[CV 4/5; 11/81] START batch_size=10, epochs=50, neuron1=4, neuron2=4............
[CV 4/5; 11/81] END batch_size=10, epochs=50, neuron1=4, neuron2=4; total time=   2.2s
[CV 5/5; 11/81] START batch_size=10, epochs=50, neuron1=4, neuron2=4............
[CV 5/5; 11/81] END batch_size=10, epochs=50, neuron1=4, neuron2=4; total time=   2.6s
[CV 1/5; 12/81] START batch_size=10, epochs=50, neuron1=4, neuron2=8.....

[CV 4/5; 20/81] END batch_size=10, epochs=100, neuron1=4, neuron2=4; total time=   5.0s
[CV 5/5; 20/81] START batch_size=10, epochs=100, neuron1=4, neuron2=4...........
[CV 5/5; 20/81] END batch_size=10, epochs=100, neuron1=4, neuron2=4; total time=   6.2s
[CV 1/5; 21/81] START batch_size=10, epochs=100, neuron1=4, neuron2=8...........
[CV 1/5; 21/81] END batch_size=10, epochs=100, neuron1=4, neuron2=8; total time=   4.6s
[CV 2/5; 21/81] START batch_size=10, epochs=100, neuron1=4, neuron2=8...........
[CV 2/5; 21/81] END batch_size=10, epochs=100, neuron1=4, neuron2=8; total time=   5.6s
[CV 3/5; 21/81] START batch_size=10, epochs=100, neuron1=4, neuron2=8...........
[CV 3/5; 21/81] END batch_size=10, epochs=100, neuron1=4, neuron2=8; total time=   4.4s
[CV 4/5; 21/81] START batch_size=10, epochs=100, neuron1=4, neuron2=8...........
[CV 4/5; 21/81] END batch_size=10, epochs=100, neuron1=4, neuron2=8; total time=   6.2s
[CV 5/5; 21/81] START batch_size=10, epochs=100, neuron1=4, neuron2

[CV 3/5; 30/81] END batch_size=20, epochs=10, neuron1=4, neuron2=8; total time=   0.8s
[CV 4/5; 30/81] START batch_size=20, epochs=10, neuron1=4, neuron2=8............
[CV 4/5; 30/81] END batch_size=20, epochs=10, neuron1=4, neuron2=8; total time=   0.5s
[CV 5/5; 30/81] START batch_size=20, epochs=10, neuron1=4, neuron2=8............
[CV 5/5; 30/81] END batch_size=20, epochs=10, neuron1=4, neuron2=8; total time=   0.7s
[CV 1/5; 31/81] START batch_size=20, epochs=10, neuron1=8, neuron2=2............
[CV 1/5; 31/81] END batch_size=20, epochs=10, neuron1=8, neuron2=2; total time=   0.6s
[CV 2/5; 31/81] START batch_size=20, epochs=10, neuron1=8, neuron2=2............
[CV 2/5; 31/81] END batch_size=20, epochs=10, neuron1=8, neuron2=2; total time=   0.5s
[CV 3/5; 31/81] START batch_size=20, epochs=10, neuron1=8, neuron2=2............
[CV 3/5; 31/81] END batch_size=20, epochs=10, neuron1=8, neuron2=2; total time=   0.5s
[CV 4/5; 31/81] START batch_size=20, epochs=10, neuron1=8, neuron2=2.....

[CV 2/5; 40/81] END batch_size=20, epochs=50, neuron1=8, neuron2=2; total time=   1.3s
[CV 3/5; 40/81] START batch_size=20, epochs=50, neuron1=8, neuron2=2............
[CV 3/5; 40/81] END batch_size=20, epochs=50, neuron1=8, neuron2=2; total time=   1.2s
[CV 4/5; 40/81] START batch_size=20, epochs=50, neuron1=8, neuron2=2............
[CV 4/5; 40/81] END batch_size=20, epochs=50, neuron1=8, neuron2=2; total time=   1.1s
[CV 5/5; 40/81] START batch_size=20, epochs=50, neuron1=8, neuron2=2............
[CV 5/5; 40/81] END batch_size=20, epochs=50, neuron1=8, neuron2=2; total time=   1.1s
[CV 1/5; 41/81] START batch_size=20, epochs=50, neuron1=8, neuron2=4............
[CV 1/5; 41/81] END batch_size=20, epochs=50, neuron1=8, neuron2=4; total time=   1.5s
[CV 2/5; 41/81] START batch_size=20, epochs=50, neuron1=8, neuron2=4............
[CV 2/5; 41/81] END batch_size=20, epochs=50, neuron1=8, neuron2=4; total time=   1.3s
[CV 3/5; 41/81] START batch_size=20, epochs=50, neuron1=8, neuron2=4.....

[CV 1/5; 50/81] END batch_size=20, epochs=100, neuron1=8, neuron2=4; total time=   2.6s
[CV 2/5; 50/81] START batch_size=20, epochs=100, neuron1=8, neuron2=4...........
[CV 2/5; 50/81] END batch_size=20, epochs=100, neuron1=8, neuron2=4; total time=   2.6s
[CV 3/5; 50/81] START batch_size=20, epochs=100, neuron1=8, neuron2=4...........
[CV 3/5; 50/81] END batch_size=20, epochs=100, neuron1=8, neuron2=4; total time=   1.9s
[CV 4/5; 50/81] START batch_size=20, epochs=100, neuron1=8, neuron2=4...........
[CV 4/5; 50/81] END batch_size=20, epochs=100, neuron1=8, neuron2=4; total time=   1.8s
[CV 5/5; 50/81] START batch_size=20, epochs=100, neuron1=8, neuron2=4...........
[CV 5/5; 50/81] END batch_size=20, epochs=100, neuron1=8, neuron2=4; total time=   1.8s
[CV 1/5; 51/81] START batch_size=20, epochs=100, neuron1=8, neuron2=8...........
[CV 1/5; 51/81] END batch_size=20, epochs=100, neuron1=8, neuron2=8; total time=   2.2s
[CV 2/5; 51/81] START batch_size=20, epochs=100, neuron1=8, neuron2

[CV 1/5; 58/81] END batch_size=40, epochs=10, neuron1=8, neuron2=2; total time=   0.5s
[CV 2/5; 58/81] START batch_size=40, epochs=10, neuron1=8, neuron2=2............
[CV 2/5; 58/81] END batch_size=40, epochs=10, neuron1=8, neuron2=2; total time=   0.5s
[CV 3/5; 58/81] START batch_size=40, epochs=10, neuron1=8, neuron2=2............
[CV 3/5; 58/81] END batch_size=40, epochs=10, neuron1=8, neuron2=2; total time=   0.5s
[CV 4/5; 58/81] START batch_size=40, epochs=10, neuron1=8, neuron2=2............
[CV 4/5; 58/81] END batch_size=40, epochs=10, neuron1=8, neuron2=2; total time=   0.5s
[CV 5/5; 58/81] START batch_size=40, epochs=10, neuron1=8, neuron2=2............
[CV 5/5; 58/81] END batch_size=40, epochs=10, neuron1=8, neuron2=2; total time=   0.5s
[CV 1/5; 59/81] START batch_size=40, epochs=10, neuron1=8, neuron2=4............
[CV 1/5; 59/81] END batch_size=40, epochs=10, neuron1=8, neuron2=4; total time=   0.5s
[CV 2/5; 59/81] START batch_size=40, epochs=10, neuron1=8, neuron2=4.....

[CV 5/5; 67/81] END batch_size=40, epochs=50, neuron1=8, neuron2=2; total time=   0.8s
[CV 1/5; 68/81] START batch_size=40, epochs=50, neuron1=8, neuron2=4............
[CV 1/5; 68/81] END batch_size=40, epochs=50, neuron1=8, neuron2=4; total time=   1.0s
[CV 2/5; 68/81] START batch_size=40, epochs=50, neuron1=8, neuron2=4............
[CV 2/5; 68/81] END batch_size=40, epochs=50, neuron1=8, neuron2=4; total time=   0.8s
[CV 3/5; 68/81] START batch_size=40, epochs=50, neuron1=8, neuron2=4............
[CV 3/5; 68/81] END batch_size=40, epochs=50, neuron1=8, neuron2=4; total time=   0.8s
[CV 4/5; 68/81] START batch_size=40, epochs=50, neuron1=8, neuron2=4............
[CV 4/5; 68/81] END batch_size=40, epochs=50, neuron1=8, neuron2=4; total time=   0.8s
[CV 5/5; 68/81] START batch_size=40, epochs=50, neuron1=8, neuron2=4............
[CV 5/5; 68/81] END batch_size=40, epochs=50, neuron1=8, neuron2=4; total time=   0.8s
[CV 1/5; 69/81] START batch_size=40, epochs=50, neuron1=8, neuron2=8.....

[CV 4/5; 77/81] END batch_size=40, epochs=100, neuron1=8, neuron2=4; total time=   1.2s
[CV 5/5; 77/81] START batch_size=40, epochs=100, neuron1=8, neuron2=4...........
[CV 5/5; 77/81] END batch_size=40, epochs=100, neuron1=8, neuron2=4; total time=   1.3s
[CV 1/5; 78/81] START batch_size=40, epochs=100, neuron1=8, neuron2=8...........
[CV 1/5; 78/81] END batch_size=40, epochs=100, neuron1=8, neuron2=8; total time=   1.5s
[CV 2/5; 78/81] START batch_size=40, epochs=100, neuron1=8, neuron2=8...........
[CV 2/5; 78/81] END batch_size=40, epochs=100, neuron1=8, neuron2=8; total time=   1.2s
[CV 3/5; 78/81] START batch_size=40, epochs=100, neuron1=8, neuron2=8...........
[CV 3/5; 78/81] END batch_size=40, epochs=100, neuron1=8, neuron2=8; total time=   1.2s
[CV 4/5; 78/81] START batch_size=40, epochs=100, neuron1=8, neuron2=8...........
[CV 4/5; 78/81] END batch_size=40, epochs=100, neuron1=8, neuron2=8; total time=   1.2s
[CV 5/5; 78/81] START batch_size=40, epochs=100, neuron1=8, neuron2

In [21]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result1.best_score_,grid_result1.best_params_))
means = grid_result1.cv_results_['mean_test_score']
stds = grid_result1.cv_results_['std_test_score']
params = grid_result1.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.7305825233459473, using {'batch_size': 10, 'epochs': 10, 'neuron1': 4, 'neuron2': 2}
0.7305825233459473,0.15435061319000673 with: {'batch_size': 10, 'epochs': 10, 'neuron1': 4, 'neuron2': 2}
0.7305825233459473,0.15435061319000673 with: {'batch_size': 10, 'epochs': 10, 'neuron1': 4, 'neuron2': 4}
0.7305825233459473,0.15435061319000673 with: {'batch_size': 10, 'epochs': 10, 'neuron1': 4, 'neuron2': 8}
0.7305825233459473,0.15435061319000673 with: {'batch_size': 10, 'epochs': 10, 'neuron1': 8, 'neuron2': 2}
0.7305825233459473,0.15435061319000673 with: {'batch_size': 10, 'epochs': 10, 'neuron1': 8, 'neuron2': 4}
0.7305825233459473,0.15435061319000673 with: {'batch_size': 10, 'epochs': 10, 'neuron1': 8, 'neuron2': 8}
0.7305825233459473,0.15435061319000673 with: {'batch_size': 10, 'epochs': 10, 'neuron1': 16, 'neuron2': 2}
0.7305825233459473,0.15435061319000673 with: {'batch_size': 10, 'epochs': 10, 'neuron1': 16, 'neuron2': 4}
0.7305825233459473,0.15435061319000673 with: {'batch_siz

In [22]:
#Creating best model

In [1]:
model = Sequential()
model.add(Flatten())
model.add(Dense(4,input_dim = 20,kernel_initializer = 'normal',activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(2,input_dim = neuron1,kernel_initializer = 'normal',activation = 'tanh'))
model.add(Dropout(0.1))
model.add(Dense(1,activation = 'softmax'))
    
adam = Adam(lr = 0.01)

NameError: name 'Sequential' is not defined

In [27]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
model.fit(X1, y, validation_split=0.35, epochs=10, batch_size=10)

Epoch 1/10
34/34 [==============================] - 0s 3ms/step - loss: 0.4939 - accuracy: 0.7589 - val_loss: 0.6026 - val_accuracy: 0.6796
Epoch 2/10
34/34 [==============================] - 0s 2ms/step - loss: 0.4980 - accuracy: 0.7589 - val_loss: 0.6003 - val_accuracy: 0.6796
Epoch 3/10
34/34 [==============================] - 0s 2ms/step - loss: 0.4867 - accuracy: 0.7589 - val_loss: 0.6008 - val_accuracy: 0.6796
Epoch 4/10
34/34 [==============================] - 0s 2ms/step - loss: 0.4902 - accuracy: 0.7589 - val_loss: 0.6005 - val_accuracy: 0.6796
Epoch 5/10
34/34 [==============================] - 0s 2ms/step - loss: 0.4775 - accuracy: 0.7589 - val_loss: 0.5983 - val_accuracy: 0.6796
Epoch 6/10
34/34 [==============================] - 0s 2ms/step - loss: 0.4767 - accuracy: 0.7589 - val_loss: 0.5965 - val_accuracy: 0.6796
Epoch 7/10
34/34 [==============================] - 0s 2ms/step - loss: 0.4720 - accuracy: 0.7589 - val_loss: 0.5966 - val_accuracy: 0.6796
Epoch 8/10
34/34 [==

In [33]:
scores = model.evaluate(X1, y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

17/17 [==============================] - 0s 1ms/step - loss: 0.4987 - accuracy: 0.7311
accuracy: 73.11%
